# chronos-t5-large

In [ ]:
import matplotlib.pyplot as plt  # ТОП прогноз самый большой(на проце очень медленный)+ праздничные дни
import numpy as np
import pandas as pd
import torch
from chronos import ChronosPipeline

# Загружаем модель напрямую из Hugging Face
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)
print(f"PyTorch использует устройство: {torch.cuda.current_device()} (GPU)" if torch.cuda.is_available() else "PyTorch использует устройство: CPU")

# Загружаем данные
data = pd.read_csv('C:/Users/bondarenkovv/Desktop/Python/Chronos/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)

# Заполняем пропущенные даты
date_range = pd.date_range(start=data['ds'].min(), end=data['ds'].max(), freq='D')
all_stores = data['store'].unique()
full_index = pd.MultiIndex.from_product([all_stores, date_range], names=['store', 'ds'])
data = data.set_index(['store', 'ds']).reindex(full_index).fillna(0).reset_index()

# Параметры прогнозирования
prediction_length = 40                    # Прогноз на 40 дней
last_n_days = 14  # Период для проверки активности магазина (последние 14 дней)

# Определяем активные магазины (те, у которых есть продажи за последние 14 дней)
last_date = data['ds'].max()
active_stores = []
for store in all_stores:
    store_data = data[(data['store'] == store) & (data['ds'] > last_date - pd.Timedelta(days=last_n_days))]
    if store_data['y'].sum() > 0:  # Если сумма продаж за последние 14 дней больше 0
        active_stores.append(store)

print(f"Активные магазины (с продажами за последние {last_n_days} дней): {len(active_stores)} из {len(all_stores)}")

# Список для хранения прогнозов
forecasts = []

# Прогноз только для активных магазинов с выводом подсказок
for store in active_stores:
    print(f"Рассчитывается прогноз для магазина: {store}")
    # Выбираем данные для текущего магазина
    store_data = data[data['store'] == store]['y'].values
    context = torch.tensor(store_data, dtype=torch.float32)  # Контекст — исторические данные
    
    # Генерируем прогноз
    forecast = pipeline.predict(context, prediction_length)  # shape: [1, num_samples, prediction_length]
    forecasts.append(forecast[0].numpy())  # Сохраняем прогноз как numpy массив
    print(f"Прогноз для магазина {store} успешно рассчитан")

# Преобразуем прогнозы в DataFrame
forecast_df = pd.DataFrame({
    'store': active_stores,
    'forecast': [np.median(f, axis=0) for f in forecasts]  # Берем медиану по сэмплам
})

# Разворачиваем прогноз по дням
forecast_days = pd.date_range(start=data['ds'].max() + pd.Timedelta(days=1), periods=prediction_length, freq='D')
forecast_expanded = forecast_df.explode('forecast').reset_index(drop=True)
forecast_expanded['ds'] = forecast_days.tolist() * len(active_stores)

# Добавляем предпраздничные дни вручную
pre_holiday_dates = pd.to_datetime([
    '2025-02-22', '2025-03-07', '2025-04-19', '2025-05-08', '2025-06-11', '2025-06-05', '2025-04-20', '2025-05-02', 
    '2025-05-03', '2025-12-28', '2025-12-29', '2025-12-30', '2025-12-31'
])
forecast_expanded['is_pre_holiday'] = forecast_expanded['ds'].isin(pre_holiday_dates).astype(int)

# Корректируем прогноз только для предпраздничных дней
forecast_expanded['forecast'] = forecast_expanded['forecast'] * (1 + 0.165 * forecast_expanded['is_pre_holiday'])

# Сохраняем результат
forecast_expanded.to_excel('Chronos_Forecast_Июль_Large.xlsx', index=False)
print("Прогноз успешно сохранен!")

# chronos-bolt-base

In [ ]:
import pandas as pd                               # ТОП прогноз(очень быстрый) надо тестить
import torch
from chronos import BaseChronosPipeline
import numpy as np

# Инициализируем пайплайн
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-base",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

print(f"PyTorch использует устройство: {torch.cuda.current_device()} (GPU)" if torch.cuda.is_available() else "PyTorch использует устройство: CPU")

# Загружаем данные
data = pd.read_csv('C:/Users/bondarenkovv/Desktop/Python/Chronos/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)

# Заполняем пропущенные даты
date_range = pd.date_range(start=data['ds'].min(), end=data['ds'].max(), freq='D')
all_stores = data['store'].unique()
full_index = pd.MultiIndex.from_product([all_stores, date_range], names=['store', 'ds'])
data = data.set_index(['store', 'ds']).reindex(full_index).fillna(0).reset_index()

# Параметры прогнозирования
prediction_length = 43  # Прогноз на 40 дней
last_n_days = 14  # Период для проверки активности магазина (последние 14 дней)

# Определяем активные магазины (те, у которых есть продажи за последние 14 дней)
last_date = data['ds'].max()
active_stores = []
for store in all_stores:
    store_data = data[(data['store'] == store) & (data['ds'] > last_date - pd.Timedelta(days=last_n_days))]
    if store_data['y'].sum() > 0:  # Если сумма продаж за последние 14 дней больше 0
        active_stores.append(store)

print(f"Активные магазины (с продажами за последние {last_n_days} дней): {len(active_stores)} из {len(all_stores)}")

# Список для хранения прогнозов
forecasts = []

# Прогноз только для активных магазинов с подсказками
for store in active_stores:
    print(f"Рассчитывается прогноз для магазина: {store}")
    store_data = data[data['store'] == store]['y'].values
    context = torch.tensor(store_data, dtype=torch.float32)
    forecast = pipeline.predict(context, prediction_length)
    forecasts.append(forecast[0].numpy())
    print(f"Прогноз для магазина {store} успешно рассчитан")

# Преобразуем прогнозы в DataFrame
forecast_df = pd.DataFrame({
    'store': active_stores,
    'forecast': [np.median(f, axis=0) for f in forecasts]
})

# Разворачиваем прогноз по дням
forecast_days = pd.date_range(start=data['ds'].max() + pd.Timedelta(days=1), periods=prediction_length, freq='D')
forecast_expanded = forecast_df.explode('forecast').reset_index(drop=True)
forecast_expanded['ds'] = forecast_days.tolist() * len(active_stores)

# Убираем отрицательные прогнозы (устанавливаем минимум 0)
forecast_expanded['forecast'] = forecast_expanded['forecast'].clip(lower=0)

# Добавляем предпраздничные дни
pre_holiday_dates = pd.to_datetime([
    '2025-02-22', '2025-03-07', '2025-04-19', '2025-05-08', '2025-06-11', '2025-06-05','2025-04-20', '2025-05-02', 
    '2025-05-03', '2025-12-28', '2025-12-29', '2025-12-30', '2025-12-31'
])
forecast_expanded['is_pre_holiday'] = forecast_expanded['ds'].isin(pre_holiday_dates).astype(int)

# Корректируем прогноз для предпраздничных дней
forecast_expanded['forecast'] = forecast_expanded['forecast'] * (1 + 0.165 * forecast_expanded['is_pre_holiday'])

# Сохраняем результат
forecast_expanded.to_excel('Chronos_Forecast_Август_Bolt_Base_обычный.xlsx', index=False)
print("Прогноз успешно сохранен!")

# Bolt-Base Expert

In [ ]:
import pandas as pd
import torch
from chronos import BaseChronosPipeline
import numpy as np

# Инициализируем пайплайн
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-base",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

print(f"PyTorch использует устройство: {torch.cuda.current_device()} (GPU)" if torch.cuda.is_available() else "PyTorch использует устройство: CPU")

# Загружаем данные
data = pd.read_csv('C:/Users/bondarenkovv/Desktop/Python/Chronos/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)

# Заполняем пропущенные даты
date_range = pd.date_range(start=data['ds'].min(), end=data['ds'].max(), freq='D')
all_stores = data['store'].unique()
full_index = pd.MultiIndex.from_product([all_stores, date_range], names=['store', 'ds'])
data = data.set_index(['store', 'ds']).reindex(full_index).fillna(0).reset_index()

# Параметры прогнозирования
prediction_length = 43  # Прогноз на 40 дней
last_n_days = 14  # Период для проверки активности магазина (последние 14 дней)

# Определяем активные магазины (те, у которых есть продажи за последние 14 дней)
last_date = data['ds'].max()
active_stores = []
for store in all_stores:
    store_data = data[(data['store'] == store) & (data['ds'] > last_date - pd.Timedelta(days=last_n_days))]
    if store_data['y'].sum() > 0:  # Если сумма продаж за последние 14 дней больше 0
        active_stores.append(store)

print(f"Активные магазины (с продажами за последние {last_n_days} дней): {len(active_stores)} из {len(all_stores)}")

# Список для хранения прогнозов
forecasts = []

# Прогноз только для активных магазинов с подсказками
for store in active_stores:
    print(f"Рассчитывается прогноз для магазина: {store}")
    store_data = data[data['store'] == store]['y'].values
    context = torch.tensor(store_data, dtype=torch.float32)
    forecast = pipeline.predict(context, prediction_length)
    forecasts.append(forecast[0].numpy())
    print(f"Прогноз для магазина {store} успешно рассчитан")

# Преобразуем прогнозы в DataFrame
forecast_df = pd.DataFrame({
    'store': active_stores,
    'forecast': [np.median(f, axis=0) for f in forecasts]
})

# Разворачиваем прогноз по дням
forecast_days = pd.date_range(start=data['ds'].max() + pd.Timedelta(days=1), periods=prediction_length, freq='D')
forecast_expanded = forecast_df.explode('forecast').reset_index(drop=True)
forecast_expanded['ds'] = forecast_days.tolist() * len(active_stores)

# Убираем отрицательные прогнозы (устанавливаем минимум 0)
forecast_expanded['forecast'] = forecast_expanded['forecast'].clip(lower=0)

# Добавляем предпраздничные дни с дифференцированными коэффициентами
holiday_multipliers = {
    '2026-02-22': 0.165,  # 23 февраля
    '2026-03-07': 0.22,   # 8 марта
    '2026-04-11': 0.2,    # Пасха
    '2026-05-02': 0.165,  # Майские праздники
    '2026-05-03': 0.165,  # Майские праздники
    '2026-05-08': 0.165,  # 9 мая
    '2026-06-11': 0.2,    # День России
    '2025-11-01': 0.05,   # День дрибидень
    '2025-12-28': 0.25,   # Новый год
    '2025-12-29': 0.3,    # Новый год
    '2025-12-30': 0.3,    # Новый год
    '2025-12-31': 0.35    # Новый год
}
pre_holiday_dates = pd.to_datetime(list(holiday_multipliers.keys()))
forecast_expanded['is_pre_holiday'] = forecast_expanded['ds'].isin(pre_holiday_dates).astype(int)
forecast_expanded['holiday_multiplier'] = forecast_expanded['ds'].map(holiday_multipliers).fillna(1.0)
forecast_expanded['forecast'] = forecast_expanded['forecast'] * forecast_expanded['holiday_multiplier']

# Сохраняем результат
forecast_expanded.to_excel('Chronos_Forecast_Август_Bolt_Base.xlsx', index=False)
print("Прогноз успешно сохранен!")

# Оценка Метрики

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Загружаем сохранённый прогноз
forecast_df = pd.read_excel('Chronos_Forecast_Июль_Bolt_Base.xlsx', sheet_name='Sheet1') # ЭТО ЧТО СРАВНИВАЕМ
forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])  # Убеждаемся, что даты в правильном формате

# Загружаем фактические данные
actual_df = pd.read_csv('C:/Users/bondarenkovv/Desktop/Python/Chronos/Actual_Sales.csv', sep=';', low_memory=False) # ЭТО С ЧЕМ СРАВНИВАЕМ
actual_df['Дата'] = pd.to_datetime(actual_df['Дата'], dayfirst=True)
actual_df = actual_df.rename(columns={
    'Дата': 'ds', 
    'Номер Магазина': 'store', 
    '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'
})
actual_df['y'] = actual_df['y'].astype(str).str.replace(',', '.').astype(float)
actual_df = actual_df.drop_duplicates(subset=['store', 'ds']).dropna()

# Объединяем прогноз и фактические данные
comparison_df = forecast_df.merge(
    actual_df[['store', 'ds', 'y']], 
    on=['store', 'ds'], 
    how='inner'  # Оставляем только совпадающие даты и магазины
)

# Проверяем, есть ли данные для сравнения
if comparison_df.empty:
    print("Нет данных для сравнения. Проверьте даты и магазины в файлах.")
else:
    # Определяем период сравнения
    min_date = comparison_df['ds'].min().strftime('%Y-%m-%d')
    max_date = comparison_df['ds'].max().strftime('%Y-%m-%d')
    num_days = (comparison_df['ds'].max() - comparison_df['ds'].min()).days + 1
    num_stores = comparison_df['store'].nunique()
    
    # Считаем суммарный прогноз и факт
    total_forecast = comparison_df['forecast'].sum()
    total_actual = comparison_df['y'].sum()
    
    # Процент отклонения (если факт = 0, избегаем деления на ноль)
    deviation_percent = ((total_forecast - total_actual) / (total_actual + 1e-10)) * 100 if total_actual != 0 else float('inf')

    # Выводим информацию о периоде и суммах
    print(f"Сравниваемый период: с {min_date} по {max_date}")
    print(f"Количество дней: {num_days}")
    print(f"Количество магазинов: {num_stores}")
    print(f"Суммарный прогноз: {total_forecast:.2f}")
    print(f"Суммарный факт: {total_actual:.2f}")
    print(f"Процент отклонения прогноза от факта: {deviation_percent:.2f}%")

    # Сравниваем прогноз с фактом
    y_true = comparison_df['y'].values
    y_pred = comparison_df['forecast'].values

    # Вычисляем метрики
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100  # Избегаем деления на 0
    smape = np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred) + 1e-10)) * 100

    # Выводим метрики
    print(f"\nMAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"SMAPE: {smape:.2f}%")

    # Сохраняем сравнение
    comparison_df.to_excel('Forecast_vs_Actual_Comparison.xlsx', index=False)
    print("Сравнение сохранено в 'Forecast_vs_Actual_Comparison.xlsx'")

In [ ]:
# После основного вывода добавляем группировку по магазинам
print("\nСравнение по магазинам:")
for store in comparison_df['store'].unique():
    store_data = comparison_df[comparison_df['store'] == store]
    store_forecast = store_data['forecast'].sum()
    store_actual = store_data['y'].sum()
    store_deviation = ((store_forecast - store_actual) / (store_actual + 1e-10)) * 100
    print(f"Магазин {store}: Прогноз = {store_forecast:.2f}, Факт = {store_actual:.2f}, Отклонение = {store_deviation:.2f}%")

# График

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Настройки для графиков
plt.style.use('seaborn')  # Стиль Seaborn для красивого отображения
plt.rcParams['figure.figsize'] = (12, 8)  # Увеличим ширину для лучшей видимости

# Фильтруем фактические данные с начала 2025 года и считаем сумму по всем магазинам
start_date = pd.to_datetime('2025-01-01')
fact_2025 = data[data['ds'] >= start_date].copy()
fact_total = fact_2025.groupby('ds')['y'].sum().reset_index()
fact_total['is_pre_holiday'] = fact_total['ds'].isin(pre_holiday_dates).astype(int)  # Добавляем метку предпраздничных дней

# Считаем суммарный прогноз по всем активным магазинам
forecast_total = forecast_expanded.groupby('ds')['forecast'].sum().reset_index()
forecast_total['is_pre_holiday'] = forecast_total['ds'].isin(pre_holiday_dates).astype(int)

# Определяем пользовательские метки с шагом 7 дней, начиная с 30 декабря 2024
start_xticks = pd.to_datetime('2024-12-30')
end_date = forecast_total['ds'].max()  # Последняя дата прогноза
xticks = pd.date_range(start=start_xticks, end=end_date, freq='7D')

# Определяем ежедневные линии для дополнительной сетки
daily_lines = pd.date_range(start=start_xticks, end=end_date, freq='D')

# Построение графика
plt.figure()
# Факт (в тысячах рублей)
plt.plot(fact_total['ds'], fact_total['y'] / 1000, label='Факт (с 2025, сумма по всем магазинам)', color='blue')
# Прогноз (в тысячах рублей)
plt.plot(forecast_total['ds'], forecast_total['forecast'] / 1000, label='Прогноз (сумма по активным магазинам)', color='green', linestyle='--')
# Отмечаем предпраздничные дни на прогнозе (в тысячах рублей)
pre_holiday_forecast = forecast_total[forecast_total['is_pre_holiday'] == 1]
plt.scatter(pre_holiday_forecast['ds'], pre_holiday_forecast['forecast'] / 1000, 
            color='red', label='Предпраздничные дни (прогноз)', zorder=5, s=50)
# Отмечаем предпраздничные дни в фактических данных (в тысячах рублей)
pre_holiday_fact = fact_total[fact_total['is_pre_holiday'] == 1]
plt.scatter(pre_holiday_fact['ds'], pre_holiday_fact['y'] / 1000, 
            color='purple', label='Предпраздничные дни (факт)', zorder=5, s=50, marker='x')  # Используем другой цвет и маркер для отличия

# Добавляем светлые серые пунктирные линии для каждого дня
plt.vlines(daily_lines, ymin=plt.gca().get_ylim()[0], ymax=plt.gca().get_ylim()[1], 
           colors='lightgrey', linestyles='dashed', linewidth=0.8, alpha=0.8, zorder=0)

plt.title('Суммарные продажи: Факт с 2025 года и прогноз')
plt.xlabel('Дата')
plt.ylabel('Сумма продаж (тыс. руб.)')  # Подпись оси Y в тысячах рублей
plt.legend()
plt.grid(True)

# Устанавливаем метки с шагом 7 дней для оси X
plt.xticks(xticks, rotation=45, ha='right')  # Метки каждые 7 дней с 30 декабря 2024
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))  # Формат даты

# Форматируем ось Y для отображения с одним знаком после запятой
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, _: f'{x:,.1f}'))

plt.tight_layout()
plt.savefig('Total_fact_2025_forecast_7days_with_daily_grid_and_fact_preholidays.png')  # Сохранение графика
plt.show()

print("График с суммарными фактическими данными с 2025 года, прогнозом, предпраздничными днями и ежедневной пунктирной сеткой построен и сохранен!")